In [1]:
import os
import torch
import pickle
import pydssp
import numpy as np
from utils.utils import *
np.set_printoptions(suppress=True, linewidth=120, formatter={'int': '{:d}'.format})

In [2]:
'''
    H = α-helix
    B = residue in isolated β-bridge
    E = extended strand, participates in β ladder
    G = 310-helix
    I = π-helix
    P = κ-helix (poly-proline II helix)
    T = hydrogen-bonded turn
    S = bend
'''
# aa = {'ALA':0, 'VAL':0, 'ILE':0, 'LEU':0, 'MET':0, 'PHE':0, 'TYR':0, 'TRP':0, 'CYS':0, 'GLY':0,
#       'PRO':0, 'SER':0, 'THR':0, 'ASN':0, 'GLN':0, 'ARG':0, 'HIS':0, 'LYS':0, 'ASP':0, 'GLU':0}
#Hydrophobic to Hydrophilic
aa = {'ILE':0, 'VAL':0, 'LEU':0, 'PHE':0, 'CYS':0, 'MET':0, 'ALA':0, 'GLY':0, 'THR':0, 'SER':0,
      'TRP':0, 'TYR':0, 'PRO':0, 'HIS':0, 'GLN':0, 'ASN':0, 'GLU':0, 'ASP':0, 'LYS':0, 'ARG':0}
aa_dict = {'A':aa.copy(), 'H':aa.copy(), 'B':aa.copy(), 'E':aa.copy(),
           'G':aa.copy(), 'I':aa.copy(), 'P':aa.copy(), 'T':aa.copy(), 'S':aa.copy()}
# LOOP HELIX CLONE
aa_dict = {0:aa.copy(), 1:aa.copy(), 2:aa.copy(), 'A':aa.copy()}

In [3]:
pdb_chain = np.load('prot_rep_2dot8.npy')

In [ ]:
count_chains = 0
atomname_dict = {'N':0,'CA':1,'C':2,'O':3}
for ii,i in enumerate(pdb_chain[:]):
    pdb = PDB(f'./PDB/{i[0]}.cif')
    chain_id = i[1].strip('\n')
    if not chain_id:
        chain_id = 'A'
    chain_idx = np.where(pdb.chain == chain_id)[0]
    res_num, res_index = np.unique(pdb.resnum[chain_idx], return_inverse=True)
    if len(res_num)<10:
        #print(i[0])
        continue
    count_chains += 1
    current_aa = np.empty((len(res_num)), dtype='<U3')
    coord = torch.zeros((len(res_num)),4,3)
    for i_idx, idx in enumerate(chain_idx):
        if pdb.atomname[idx] in ('N', 'CA', 'C', 'O'):
            coord[res_index[i_idx],atomname_dict[pdb.atomname[idx]]] = torch.Tensor(pdb.coords[idx])
            current_aa[res_index[i_idx]] = pdb.resname[chain_idx][i_idx]
    dssp = pydssp.assign(coord, out_type='index')
    for aa_i, amino_acid in enumerate(current_aa):
        if amino_acid not in aa.keys():
            continue
        aa_dict['A'][amino_acid] += 1
        aa_dict[int(dssp[aa_i])][amino_acid] += 1
    if not ii%100:
        print(ii)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700


In [ ]:
count_chains

In [5]:
count_chains

1178

In [ ]:
# Save to file
with open("aa_info_proteins_res_2dot8.pkl", "wb") as f:
    pickle.dump(aa_dict, f)

# # Load back
# with open("aa_info_proteins.pkl", "rb") as f:
#     aa_dict = pickle.load(f)

# with open("aa_info_membrane_proteins.pkl", "rb") as f:
#     aa_dict = pickle.load(f)

In [ ]:
aa_arr = np.array([[aa_dict[i][j] for j in list(aa_dict[0].keys())] for i in list(aa_dict.keys())])
aa_prop_arr = aa_arr.copy().astype(float)
for i in range(len(aa_prop_arr)):
    aa_prop_arr[i] /= aa_arr[i].sum()
for i in range(len(aa_prop_arr)-1):
    aa_prop_arr[i] /= aa_prop_arr[-1]

In [ ]:
for ii, ss in enumerate(('loop', 'helix', 'beta_strand', 'aa_probability')):
    with open(f'./latex/data/protein_{ss}_propensity_res_2dot8.dat', 'w') as f:
        f.write(f'AA Propensity\n')
        for aa_i, aa_n in enumerate(list(aa.keys())):
            f.write(f'{aa_n} {aa_prop_arr[ii][aa_i]}\n')                

In [8]:
import numpy as np
from scipy import stats
import pandas as pd

# Get amino acid names
amino_acids = list(aa_dict[0].keys())

# Create arrays
aa_arr = np.array([[aa_dict[i][j] for j in amino_acids] for i in [0, 1, 2, 'A']])
print("Data shape:", aa_arr.shape)
print("Amino acids:", amino_acids)

# Method 1: Chi-square test comparing distribution across categories 0, 1, 2
def chi_square_test():
    """Test if amino acid distribution differs significantly across categories 0, 1, 2"""
    print("\n=== Chi-square test for each amino acid ===")
    
    # Use only categories 0, 1, 2 (not 'A' which seems to be total)
    observed = aa_arr[:3, :]  # shape: (3 categories, 20 amino acids)
    
    p_values_chi2 = []
    chi2_stats = []
    
    for i, aa in enumerate(amino_acids):
        # For each amino acid, test if its distribution across categories differs from expected
        observed_aa = observed[:, i]  # counts for this AA in categories 0, 1, 2
        
        # Chi-square goodness of fit test (uniform distribution as null hypothesis)
        chi2_stat, p_val = stats.chisquare(observed_aa)
        
        chi2_stats.append(chi2_stat)
        p_values_chi2.append(p_val)
        
        print(f"{aa}: chi2={chi2_stat:.2f}, p={p_val:.2e}")
    
    return p_values_chi2, chi2_stats

# Method 2: Comparing each category against expected proportions
def proportion_tests():
    """Test if proportion of each amino acid in categories 0,1,2 differs from category A"""
    print("\n=== Proportion tests against reference (category A) ===")
    
    # Use category 'A' as reference
    total_ref = aa_arr[3, :]  # category 'A'
    
    p_values_prop = {0: [], 1: [], 2: []}
    
    for cat in [0, 1, 2]:
        print(f"\nCategory {cat} vs Reference A:")
        observed_cat = aa_arr[cat, :]
        total_observed = observed_cat.sum()
        total_reference = total_ref.sum()
        
        for i, aa in enumerate(amino_acids):
            # Two-proportion z-test
            x1, n1 = observed_cat[i], total_observed  # observed category
            x2, n2 = total_ref[i], total_reference     # reference category
            
            # Calculate proportions
            p1 = x1 / n1
            p2 = x2 / n2
            
            # Pooled proportion for z-test
            p_pool = (x1 + x2) / (n1 + n2)
            se = np.sqrt(p_pool * (1 - p_pool) * (1/n1 + 1/n2))
            
            if se > 0:
                z_stat = (p1 - p2) / se
                p_val = 2 * (1 - stats.norm.cdf(abs(z_stat)))  # two-tailed
            else:
                p_val = 1.0
            
            p_values_prop[cat].append(p_val)
            print(f"{aa}: p1={p1:.4f}, p2={p2:.4f}, z={z_stat:.2f}, p={p_val:.2e}")
    
    return p_values_prop

# Method 3: Multinomial test using scipy
def multinomial_tests():
    """Multinomial test for each amino acid across categories"""
    print("\n=== Multinomial tests ===")
    
    p_values_multi = []
    
    for i, aa in enumerate(amino_acids):
        observed = aa_arr[:3, i]  # categories 0, 1, 2
        
        # Use equal probabilities as null hypothesis
        expected_probs = [1/3, 1/3, 1/3]
        
        # Multinomial test using chi-square approximation
        chi2_stat = np.sum((observed - np.sum(observed) * np.array(expected_probs))**2 / 
                          (np.sum(observed) * np.array(expected_probs)))
        
        # Degrees of freedom = categories - 1
        p_val = 1 - stats.chi2.cdf(chi2_stat, df=2)
        
        p_values_multi.append(p_val)
        print(f"{aa}: observed={observed}, chi2={chi2_stat:.2f}, p={p_val:.2e}")
    
    return p_values_multi

# Method 4: Pairwise comparisons between categories
def pairwise_comparisons():
    """Pairwise proportion tests between categories"""
    print("\n=== Pairwise comparisons between categories ===")
    
    comparisons = [(0, 1), (0, 2), (1, 2)]
    pairwise_results = {}
    
    for cat1, cat2 in comparisons:
        print(f"\nComparing category {cat1} vs category {cat2}:")
        pairwise_results[f"{cat1}_vs_{cat2}"] = []
        
        obs1 = aa_arr[cat1, :]
        obs2 = aa_arr[cat2, :]
        total1 = obs1.sum()
        total2 = obs2.sum()
        
        for i, aa in enumerate(amino_acids):
            # Two-proportion z-test
            x1, n1 = obs1[i], total1
            x2, n2 = obs2[i], total2
            
            p1 = x1 / n1
            p2 = x2 / n2
            
            # Pooled proportion
            p_pool = (x1 + x2) / (n1 + n2)
            se = np.sqrt(p_pool * (1 - p_pool) * (1/n1 + 1/n2))
            
            if se > 0:
                z_stat = (p1 - p2) / se
                p_val = 2 * (1 - stats.norm.cdf(abs(z_stat)))
            else:
                p_val = 1.0
            
            pairwise_results[f"{cat1}_vs_{cat2}"].append(p_val)
            print(f"{aa}: p1={p1:.4f}, p2={p2:.4f}, p={p_val:.2e}")
    
    return pairwise_results

# Run all methods
if __name__ == "__main__":
    # Method 1: Chi-square tests
    p_vals_chi2, chi2_stats = chi_square_test()
    
    # Method 2: Proportion tests against reference
    p_vals_prop = proportion_tests()
    
    # Method 3: Multinomial tests
    p_vals_multi = multinomial_tests()
    
    # Method 4: Pairwise comparisons
    pairwise_results = pairwise_comparisons()
    
    # Create summary DataFrame
    print("\n=== SUMMARY TABLE ===")
    
    results_df = pd.DataFrame({
        'Amino_Acid': amino_acids,
        'Chi2_pvalue': p_vals_chi2,
        'Multinomial_pvalue': p_vals_multi,
        'Cat0_vs_RefA_pvalue': p_vals_prop[0],
        'Cat1_vs_RefA_pvalue': p_vals_prop[1],
        'Cat2_vs_RefA_pvalue': p_vals_prop[2],
        'Cat0_vs_Cat1_pvalue': pairwise_results['0_vs_1'],
        'Cat0_vs_Cat2_pvalue': pairwise_results['0_vs_2'],
        'Cat1_vs_Cat2_pvalue': pairwise_results['1_vs_2']
    })
    
    print(results_df.to_string(index=False, float_format='%.2e'))
    
    # Apply multiple testing correction (Bonferroni)
    print("\n=== Multiple Testing Correction (Bonferroni) ===")
    
    from statsmodels.stats.multitest import multipletests
    
    # Correct chi-square p-values
    rejected, p_corrected, _, _ = multipletests(p_vals_chi2, method='bonferroni')
    
    correction_df = pd.DataFrame({
        'Amino_Acid': amino_acids,
        'Chi2_pvalue_raw': p_vals_chi2,
        'Chi2_pvalue_corrected': p_corrected,
        'Significant_after_correction': rejected
    })
    
    print(correction_df.to_string(index=False, float_format='%.2e'))

Data shape: (4, 20)
Amino acids: ['ALA', 'VAL', 'ILE', 'LEU', 'MET', 'PHE', 'TYR', 'TRP', 'CYS', 'GLY', 'PRO', 'SER', 'THR', 'ASN', 'GLN', 'ARG', 'HIS', 'LYS', 'ASP', 'GLU']

=== Chi-square test for each amino acid ===
ALA: chi2=48996.16, p=0.00e+00
VAL: chi2=9115.19, p=0.00e+00
ILE: chi2=6267.86, p=0.00e+00
LEU: chi2=27677.46, p=0.00e+00
MET: chi2=4055.30, p=0.00e+00
PHE: chi2=441.59, p=1.29e-96
TYR: chi2=296.79, p=3.57e-65
TRP: chi2=694.34, p=1.68e-151
CYS: chi2=948.00, p=1.39e-206
GLY: chi2=131575.19, p=0.00e+00
PRO: chi2=97546.16, p=0.00e+00
SER: chi2=30928.58, p=0.00e+00
THR: chi2=11289.64, p=0.00e+00
ASN: chi2=47731.57, p=0.00e+00
GLN: chi2=16892.58, p=0.00e+00
ARG: chi2=14646.86, p=0.00e+00
HIS: chi2=6401.87, p=0.00e+00
LYS: chi2=21202.99, p=0.00e+00
ASP: chi2=59123.56, p=0.00e+00
GLU: chi2=42018.47, p=0.00e+00

=== Proportion tests against reference (category A) ===

Category 0 vs Reference A:
ALA: p1=0.0649, p2=0.0828, z=-66.64, p=0.00e+00
VAL: p1=0.0456, p2=0.0699, z=-99.19, 

ModuleNotFoundError: No module named 'statsmodels'

In [9]:
results_df

,Amino_Acid,Chi2_pvalue,Multinomial_pvalue,Cat0_vs_RefA_pvalue,Cat1_vs_RefA_pvalue,Cat2_vs_RefA_pvalue,Cat0_vs_Cat1_pvalue,Cat0_vs_Cat2_pvalue,Cat1_vs_Cat2_pvalue
0,ALA,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,1.020630e-09,0.000000e+00
1,VAL,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000e+00
2,ILE,0.000000e+00,0.0,0.000000e+00,4.779968e-03,0.000000,0.0,0.000000e+00,0.000000e+00
3,LEU,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000e+00
4,MET,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000055,0.0,0.000000e+00,0.000000e+00
5,PHE,1.286445e-96,0.0,0.000000e+00,1.678374e-06,0.000000,0.0,0.000000e+00,0.000000e+00
6,TYR,3.568000e-65,0.0,0.000000e+00,3.041405e-07,0.000000,0.0,0.000000e+00,0.000000e+00
7,TRP,1.683968e-151,0.0,0.000000e+00,3.132827e-12,0.000000,0.0,0.000000e+00,0.000000e+00
8,CYS,1.394650e-206,0.0,5.016331e-02,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000e+00
9,GLY,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000e+00
